In [2]:
import openvino.runtime as ov
import cv2
import numpy as np
import tensorflow as tf
import yaml

import os
import time

In [3]:
core = ov.Core()

In [4]:
compiled_model = core.compile_model("landmark_model/mobilefacenet.xml", "AUTO")

In [5]:
infer_request = compiled_model.create_infer_request()

In [6]:
with open("config.yaml", "r") as f:
    config = yaml.load(f, Loader=yaml.loader.FullLoader)
video = cv2.VideoCapture("images/NTHUDDD_dataset/Training_Evaluation_Dataset/Training Dataset/032/noglasses/nonsleepyCombination.avi")
valid, image = video.read()
for i in range(0, 300):
    valid, image = video.read()

# cv2.imshow('', image)
# cv2.waitKey()

In [7]:
modelFile = "res10_300x300_ssd_iter_140000.caffemodel"
configFile = "deploy.prototxt"
net = cv2.dnn.readNetFromCaffe(configFile, modelFile)
h, w = image.shape[:2]
blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 1.0,
(300, 300), (104.0, 117.0, 123.0))
net.setInput(blob)
faces = net.forward()
#to draw faces on image
facebox = ()
for i in range(faces.shape[2]):
    confidence = faces[0, 0, i, 2]
    if confidence > 0.5:
        box = faces[0, 0, i, 3:7] * np.array([w, h, w, h])
        (x, y, x1, y1) = box.astype("int")
        facebox = (x, y, x1, y1)
        # cv2.rectangle(image, (x, y), (x1, y1), (0, 0, 255), 2)

face_img = image[facebox[1]: facebox[3],
                facebox[0]: facebox[2]]

face_height, face_width, _ = face_img.shape

# cv2.imshow("", face_img)
# cv2.waitKey()

In [8]:
image_float = cv2.resize(face_img.copy(), dsize=(112, 112), interpolation=cv2.INTER_AREA).astype(np.float32)
image_float = image_float / 255.0
image_tensor = image_float.transpose((2, 0, 1))
image_tensor = tf.expand_dims(image_tensor, axis=0)
image_float = np.asarray(image_tensor)
#image_float = np.ones([1,3,32,32], np.float32)

print(image_float.shape)

input_tensor = ov.Tensor(array=image_float, shared_memory=True)
infer_request.set_input_tensor(input_tensor)

infer_request.start_async()
infer_request.wait()

output = infer_request.get_output_tensor()
landmarks = output.data.reshape(-1, 2)

for landmark in landmarks:
    x = landmark[0] * face_width + facebox[0]
    y = landmark[1] * face_height + facebox[1]
    cv2.circle(image, (int(x), int(y)), 1, (0, 255, 0), -1, cv2.LINE_AA)

cv2.imshow('', image)
cv2.waitKey()

(1, 3, 112, 112)


-1

In [5]:
prediction_dict = {1: "open", 0: "closed"}

In [ ]:
sigma_s = 3
sigma_r = 0.1
file = "output_eyes/eye1.jpg"
image = cv2.imread(file)
cv2.imshow('', image)
image = cv2.detailEnhance(image, sigma_s=sigma_s, sigma_r=sigma_r)
cv2.imshow('a', image)
cv2.waitKey()

In [ ]:
from PIL import Image
import cv2
def obtain_samples_from_dir(dataset_path) -> dict:
    images = []
    for filename in os.listdir(dataset_path):
        file = os.path.join(dataset_path, filename)
        if os.path.isdir(file):
            images = images + obtain_samples_from_dir(file)
        elif os.path.isfile(file) and filename[-4:] == ".png":
            label = filename.split("_")[4]
            images.append((file, label))
    
    return images

In [ ]:
dataset_path = "../mrlEyes_2018_01/"
data = obtain_samples_from_dir(dataset_path)

In [6]:
num_hits = 0
for file, label in data:
    image = cv2.imread(file).astype(np.float32)
    # image = cv2.detailEnhance(image, sigma_s=sigma_s, sigma_r=sigma_r)
    # image = sr.upsample(image)
    #image = cv2.detailEnhance(image, sigma_s=sigma_s, sigma_r=sigma_r)
    #image_float = cv2.normalize(image, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    image_float = cv2.resize(image, dsize=(32, 32), interpolation=cv2.INTER_AREA)
    image_tensor = tf.convert_to_tensor(image_float, dtype=tf.float32)
    image_tensor = tf.transpose(image_tensor, [2, 0, 1])
    image_tensor = tf.expand_dims(image_tensor, axis=0)
    image_float = np.asarray(image_tensor)
    #image_float = np.ones([1,3,32,32], np.float32)

    input_tensor = ov.Tensor(array=image_float, shared_memory=True)
    infer_request.set_input_tensor(input_tensor)

    infer_request.start_async()
    infer_request.wait()

    output = infer_request.get_output_tensor()
    output_buffer = output.data
    #print(file)
    #print(output_buffer)
    prediction = int(np.argmax(np.array(output_buffer).squeeze(), axis=0))
    #print(prediction_dict[prediction])
    if prediction == int(label):
        print("hit!!!")
        num_hits += 1
    else:
        print("fail!!!")

print(f"NUM_CLOSED: {num_hits/len(data)}")

NameError: name 'data' is not defined

In [7]:
closed = "s0001_00001_0_0_0_0_0_01.png"
file = "images/mrlEyes_2018_01/s0001/s0001_03094_0_1_1_2_0_01.png"

best_num_closed = -1
best_sigma_s = -1
best_sigma_r = -1
# for sigma_s in range(0, 200, 1):
#     for sigma_r2 in range(0, 10, 1):
        # sigma_r = sigma_r2 * 0.1
num_closed = 0
path = "output_eyes/"
for filename in os.listdir(path):
    file = os.path.join(path, filename)
    #file = "eye.jpg"
    image = cv2.imread(file).astype(np.float32)	
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # image = cv2.detailEnhance(image, sigma_s=sigma_s, sigma_r=sigma_r)
    # image = sr.upsample(image)
    #image = cv2.detailEnhance(image, sigma_s=sigma_s, sigma_r=sigma_r)
    #image_float = cv2.normalize(image, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    image_float = cv2.resize(image, dsize=(32, 32), interpolation=cv2.INTER_AREA)
    image_tensor = tf.convert_to_tensor(image_float, dtype=tf.float32)
    image_tensor = np.stack((image_tensor,)*3, axis=-1)
    image_tensor = tf.transpose(image_tensor, [2, 0, 1])
    image_tensor = tf.expand_dims(image_tensor, axis=0)
    image_float = np.asarray(image_tensor)
    print(image_float.shape)
    #image_float = np.ones([1,3,32,32], np.float32)

    input_tensor = ov.Tensor(array=image_float, shared_memory=True)
    infer_request.set_input_tensor(input_tensor)

    infer_request.start_async()
    infer_request.wait()

    output = infer_request.get_output_tensor()
    output_buffer = output.data
    print(filename)
    print(output_buffer)
    prediction = int(np.argmax(np.array(output_buffer).squeeze(), axis=0))
    print(prediction_dict[prediction])
    if prediction_dict[prediction] == "closed":
        num_closed += 1

print(f"NUM_CLOSED: {num_closed}")
    # if num_closed > best_num_closed:
    #     best_sigma_s = sigma_s
    #     best_sigma_r = sigma_r
    #     best_num_closed = num_closed

# print(f"BEST NUM CLOSED: {best_num_closed}")
# print(f"BEST SIGMA_S: {best_sigma_s}")
# print(f"BEST SIGMA_R: {best_sigma_r}")

(1, 3, 32, 32)
eye0.jpg
[[[[8.40594e-24]]

  [[1.00000e+00]]]]
open
(1, 3, 32, 32)
eye1.jpg
[[[[5.7725418e-22]]

  [[9.9999994e-01]]]]
open
(1, 3, 32, 32)
eye10.jpg
[[[[1.41768195e-24]]

  [[1.00000000e+00]]]]
open
(1, 3, 32, 32)
eye100.jpg
[[[[6.449424e-17]]

  [[1.000000e+00]]]]
open
(1, 3, 32, 32)
eye1000.jpg
[[[[4.0270784e-15]]

  [[9.9999994e-01]]]]
open
(1, 3, 32, 32)
eye1001.jpg
[[[[3.5063334e-11]]

  [[1.0000000e+00]]]]
open
(1, 3, 32, 32)
eye1002.jpg
[[[[3.273590e-05]]

  [[9.999672e-01]]]]
open
(1, 3, 32, 32)
eye1003.jpg
[[[[0.70227927]]

  [[0.2977207 ]]]]
closed
(1, 3, 32, 32)
eye1004.jpg
[[[[2.8086927e-07]]

  [[9.9999970e-01]]]]
open
(1, 3, 32, 32)
eye1005.jpg
[[[[1.0783318e-06]]

  [[9.9999893e-01]]]]
open
(1, 3, 32, 32)
eye1006.jpg
[[[[1.3179494e-10]]

  [[1.0000000e+00]]]]
open
(1, 3, 32, 32)
eye1007.jpg
[[[[1.6396762e-11]]

  [[1.0000000e+00]]]]
open
(1, 3, 32, 32)
eye1008.jpg
[[[[1.323356e-10]]

  [[1.000000e+00]]]]
open
(1, 3, 32, 32)
eye1009.jpg
[[[[6.293583e-11]]


In [ ]:
import matplotlib.pyplot as plt
file = "output_eyes/eye1.jpg"
img = cv2.imread(file)

sr = cv2.dnn_superres.DnnSuperResImpl_create()

path = "EDSR_x4.pb"

sr.readModel(path)

sr.setModel("edsr",4)

result = sr.upsample(img)

# Resized image
resized = cv2.resize(img,dsize=None,fx=4,fy=4)

plt.figure(figsize=(12,8))
plt.subplot(1,3,1)
# Original image
plt.imshow(img[:,:,::-1])
plt.subplot(1,3,2)
# SR upscaled
plt.imshow(result[:,:,::-1])
plt.subplot(1,3,3)
# OpenCV upscaled
plt.imshow(resized[:,:,::-1])
plt.show()